# Imports

In [1]:
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import wrangle as w
import explore as e
import modeling as m

# Acquire

* Data aquired from [Kaggle](https://www.kaggle.com/satpreetmakhija/netflix-movies-and-tv-shows-2021) on 2/04/2022
* Each observation represents on movie or film series 
* Because the distinction between a single movie an a multi-part series is not relevant to this study I will be referring to each as a movie for the duration of this study
* The original data set had 5967 rows and 13 columns

# Prepare

* The dataframe retained 5494 and 2 columns after preperarion
* Data was prepared using the fillowing steps:
    * Drop all columns other than 'description' and 'genre'
    * Lowercased text in both columns
    * Lowercased column names
    * Prepared text in 'description' for exploration using the following steps:
      * Removed all non-ascii and special characters form the text
      * Tokenized the words in the text
      * Lemmatized the words in the text
      * Removed stopwords
    * my goal for genre was to insure that each film only represented one genre after preperation
    * Prepared genre text using the following steps:
      * Removed text indicating if a film is a standalone movie or series
      * Created new column of curated list of genres       
      * The following genres were removed because they do not refer the films theam:
          * international
          * korean
          * teen
          * british
          * spanish-language 
          * independent
          * anime
          * kids
          * children & family    
          * classic & cult
          * classic
          * cult
      * The following genres were removed due to low representation
          * lgbtq
          * sci-fi & fantasy
          * sports
          * faith & spirituality
          * stand-up comedy & talk
          * mysteries
      * Dropped rows with empty genre lists after above genres were removed
      * Merged films with docuseries, documentary, or documentaries in it's genre list together into one genre documentaries
      * Merged together films with the following geres together under that genre alone
          * reality
          * music & musicals
          * crime
          * horror
      * Removed dramas from genre lists that contained three or more genes
      * Fused together the following genres to create new genres
          * romantic and comedies are now romantic comedies
          * dramas and comedies are now dramitic comedies
          * action & adventure and comedies are now action & adventure comedies
      * Removed drama from films that had other genres
      * Dropped rows that had more than one genre    

In [4]:
df = w.get_my_movie_data()

# Explore